In [27]:
import anthropic
import os

path = '/work/bioinformatics/s229618/api_keys/claude_key.txt'

with open(path, 'r') as file:
    key = file.read()

client = anthropic.Anthropic(
    api_key=key,
)
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0,
    system="Respond only in pirate speak.",
    messages=[
        {"role": "user", "content": "Hi Claude."}
    ]
)
print(message.content)

[TextBlock(text="Ahoy matey! 'Tis a fine day to be sailin' the high seas o' conversation, isn't it? What winds be blowin' ye way on this voyage?", type='text')]


In [28]:
transcript_id = '10_0991_331330'
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer

def read_transcript_from_id(transcript_id):

    path_to_data_folder = '/archive/shared/sim_center/shared/ameer/'
    # lookinto this dictionary to find the path
    # can also manually create the path and it would be faster but not by much
    merged_lookup = pd.read_csv(path_to_data_folder + 'grade_lookupv5.csv')
    transcript = ''

    path = merged_lookup[merged_lookup.id == transcript_id].path.iloc[0]
    with open(path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line != '\n':
            tok_line = line.split(' ')
            for i in range(len(tok_line)):
                transcript += ' ' + tok_line[i]


    return transcript

In [29]:
transcript = read_transcript_from_id(transcript_id)

In [35]:
summary = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=4096,
    temperature=0,
    system="Please summarize the following text in one paragraph. 100 words. Do not add any information that is not in the text.",
    messages=[
        {"role": "user", "content": transcript}
    ]
)
print(message.content)

[TextBlock(text='Here is a one paragraph, 100 word summary of the text:\n\nThe patient, Mr. Pimbleton, is a 55-year-old retired man experiencing nausea, vomiting undigested food, and mild abdominal pain for the past two days. He has no fever, bloody vomit, or significant medical history besides hypertension, osteoarthritis, and chronic back pain. The medical student performed a physical exam, finding no concerning abnormalities. Potential causes discussed include esophageal constriction, obstruction, or viral gastroenteritis. The plan is to pursue further testing like a barium swallow, CT scan, or endoscopy to investigate and rule out serious underlying conditions.', type='text')]


In [45]:
summary.content[0].text

'Here is a one paragraph, 100 word summary of the text:\n\nThe patient, Mr. Pimbleton, is a 55-year-old retired man experiencing nausea, vomiting undigested food, and mild abdominal pain for the past two days. He has no fever, bloody vomit, or significant medical history besides hypertension, osteoarthritis, and chronic back pain. The medical student performed a physical exam, finding no concerning abnormalities. Potential causes discussed include esophageal constriction, obstruction, or viral gastroenteritis. The plan is to pursue further testing like a barium swallow, CT scan, or endoscopy to investigate and rule out serious underlying conditions.'

In [46]:
diarization = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=4096,
    temperature=0,
    system='''The following text represents an audio transcript between a medical student and a patient, with the summary included below.
            The student is the one who will be playing the role of doctor, asking questions about the patient's condition and symptoms.
            Please diarize the following transcript in order to indicate who speaks when, using the format in the following example, with the label "Student" for the medical student and "Patient" for the patient (not their names):

            Student: Hello, how are you today?

            Patient: I am feeling sick. 

            Student: Oh no, how long have you felt sick? 

            Patient: About two days. 

            Separate speaker turns by an extra new line, as above ('\n\n'). The lines do not always have to be alternating labels. There may be consecutive lines from one speaker. Do not use any speaker labels for the two participants other than "Student" and "Patient."

            The transcript may contain some instructions for the student, coming from a third-party speaker. Please label this segment "Instructions: ". 
            If there are no instructions in the transcript, disregard this label. 

            IMPORTANT: Do not remove or add any words to the transcript other than the speaker labels. Also, do not add or remove any punctuation or change any spellings.''',
    messages=[
        {"role": "user", "content": 'summary: ' + summary.content[0].text + '\n\ntranscript: ' + transcript}
    ]
)
print(diarization.content)

[TextBlock(text="Student: Hello, Mr. Pimbleton. Hi, my name is Coleman. I'm a medical student at UT Southwestern. I am here to ask you some questions. I just want to know how you are feeling today.\n\nPatient: I've been feeling sick to my stomach. Now I throw one up, I feel not good.\n\nStudent: Okay, when did this start?\n\nPatient: It started getting nauseous. I was feeling nausea. I had a stomachache. I felt nause up. I am not good.\n\nStudent: Okay, when did this start?\n\nPatient: I started getting nauseous two days ago. Threw up the first time yesterday after breakfast, about 20 minutes after I ate.\n\nStudent: Any other episodes of vomiting?\n\nPatient: Yeah, I threw up three or four times yesterday and then after breakfast today about five minutes after I ate.\n\nStudent: Okay. And what does your vomit look like?\n\nPatient: It's just small bits of undigested food, whatever I eat.\n\nStudent: Any blood in it?\n\nPatient: No.\n\nStudent: No? No Bile?\n\nPatient: No\n\nStudent: J

In [48]:
print(diarization.content[0].text)

Student: Hello, Mr. Pimbleton. Hi, my name is Coleman. I'm a medical student at UT Southwestern. I am here to ask you some questions. I just want to know how you are feeling today.

Patient: I've been feeling sick to my stomach. Now I throw one up, I feel not good.

Student: Okay, when did this start?

Patient: It started getting nauseous. I was feeling nausea. I had a stomachache. I felt nause up. I am not good.

Student: Okay, when did this start?

Patient: I started getting nauseous two days ago. Threw up the first time yesterday after breakfast, about 20 minutes after I ate.

Student: Any other episodes of vomiting?

Patient: Yeah, I threw up three or four times yesterday and then after breakfast today about five minutes after I ate.

Student: Okay. And what does your vomit look like?

Patient: It's just small bits of undigested food, whatever I eat.

Student: Any blood in it?

Patient: No.

Student: No? No Bile?

Patient: No

Student: Just the chunks of food?

Patient: No

Student

In [49]:
path = '/archive/shared/sim_center/shared/annie/claude/'
with open(path + transcript_id + ".txt", "w") as outfile:
    outfile.write('ID: ' + transcript_id + '\n\n' + diarization.content[0].text)